## CRF model

Modelo CRF para ato de aposentadoria (com dados rotulados manualmente)

In [20]:
import pandas as pd
import sklearn_crfsuite

Load csv with text and labels

In [21]:
data = pd.read_csv('labeled.csv')
x = []
y = []
for row in range(len(data)):
    if pd.notna(data['text'][row]) and pd.notna(data['labels'][row]):
        x.append(data['text'][row].split())
        y.append(data['labels'][row].split())
len(x), len(y)

(5162, 5162)

In [22]:
x[0]

['341-6,',
 'no',
 'cargo',
 'de',
 'Tecnico',
 'em',
 'Politicas',
 'Publicas',
 'e',
 'Gestao',
 'Governamental,',
 'Classe',
 'Unica,',
 'Padrao',
 'X,',
 'do',
 'Quadro',
 'de',
 'Pessoal',
 'do',
 'Distrito',
 'Federal,',
 'termos',
 'do',
 'artigo',
 '40,',
 '1o,',
 'inciso',
 'III,',
 'alinea',
 '"b",',
 'e',
 '3o,',
 '8o',
 'e',
 '17,',
 'da',
 'Constituicao',
 'da',
 'Republica',
 'Federativa',
 'do',
 'Brasil,',
 'na',
 'redacao',
 'dada',
 'pela',
 'Emenda',
 'Constitucional',
 'no',
 '41,',
 'de',
 '19/12/2003,',
 'e',
 'artigos',
 '46',
 'e',
 '51',
 'da',
 'Lei',
 'Complementar',
 'no',
 '769,',
 'de',
 '30/06/2008.',
 'Lotacao:',
 'Administracao',
 'Regional',
 'de',
 'Taguatinga.',
 'Processo',
 'SEI',
 'n.o',
 '00132-00005298/2018-',
 '77.',
 'CONCEDER,',
 'aposentadoria',
 'voluntaria',
 'integral,',
 'a',
 'servidora',
 'MARIA',
 'DE',
 'JESUS',
 'BARBOSA',
 'SANTOS,',
 'matricula',
 'n.o',
 '31.727-6,',
 'no',
 'cargo',
 'de',
 'Analista',
 'em',
 'Politicas',
 'Pub

Create dictionary feature for each word in each sequence in x

In [23]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

Separate train and test splits (in order)

In [24]:
import math
split_idx = math.floor(0.8*len(x))
x_train = x[0:split_idx]
y_train = y[0:split_idx]
x_test = x[split_idx:]
y_test = y[split_idx:]
len(x_train),  len(x_test), len(x)


(4129, 1033, 5162)

In [25]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm = 'l2sgd',
    c2=1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(x_train, y_train)

In [26]:
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

/home/jose/anaconda3/envs/ner/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.5114313469557288

In [27]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=classes, digits=3
))

/home/jose/anaconda3/envs/ner/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass labels=['B-QUADRO', 'I-QUADRO', 'B-ATO', 'I-ATO', 'B-NOME_ATO', 'I-NOME_ATO', 'B-CARGO', 'I-CARGO', 'B-CLASSE', 'I-CLASSE', 'B-PADRAO', 'I-PADRAO', 'B-FUND_LEGAL', 'I-FUND_LEGAL', 'B-EMPRESA_ATO', 'I-EMPRESA_ATO', 'B-PROCESSO', 'I-PROCESSO', 'B-COD_MATRICULA_ATO', 'I-COD_MATRICULA_ATO'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/home/jose/anaconda3/envs/ner/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jose/anaconda3/envs/ner/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetri

                     precision    recall  f1-score   support

           B-QUADRO      0.240     0.006     0.012      1017
           I-QUADRO      0.373     0.008     0.015      5348
              B-ATO      0.800     1.000     0.889       963
              I-ATO      0.805     1.000     0.892       808
         B-NOME_ATO      0.678     0.965     0.796      1032
         I-NOME_ATO      0.675     0.970     0.796      3290
            B-CARGO      0.464     0.364     0.408      1030
            I-CARGO      0.447     0.382     0.412      3202
           B-CLASSE      0.522     0.519     0.521       520
           I-CLASSE      0.522     0.517     0.520       522
           B-PADRAO      0.496     0.359     0.417      1008
           I-PADRAO      0.496     0.359     0.417      1008
       B-FUND_LEGAL      0.770     0.661     0.711       386
       I-FUND_LEGAL      0.741     0.625     0.678     11720
      B-EMPRESA_ATO      0.350     0.069     0.115       508
      I-EMPRESA_ATO    